In [1]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.width=120

In [2]:
sms=get_data('sms-spam')
index=['sms{}{}'.format(i,'!'*j) for (i,j) in zip(range(len(sms)),sms.spam)]

In [3]:
sms.index=index

In [4]:
sms.head(6)

,spam,text
sms0,0,"Go until jurong point, crazy.. Available only ..."
sms1,0,Ok lar... Joking wif u oni...
sms2!,1,Free entry in 2 a wkly comp to win FA Cup fina...
sms3,0,U dun say so early hor... U c already then say...
sms4,0,"Nah I don't think he goes to usf, he lives aro..."
sms5!,1,FreeMsg Hey there darling it's been 3 week's n...


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf=TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs=tfidf.fit_transform(raw_documents=sms.text).toarray()

In [6]:
len(tfidf.vocabulary_)

9232

In [7]:
tfidf_docs=pd.DataFrame(tfidf_docs)


In [8]:
tfidf_docs=tfidf_docs-tfidf_docs.mean()


In [9]:
tfidf_docs.head(2)

,0,1,2,3,4,5,6,7,8,9,...,9222,9223,9224,9225,9226,9227,9228,9229,9230,9231
0,-0.025643,-0.00584,-0.000228,-0.000053,-0.000156,-0.000943,-0.000463,-0.006695,-0.004035,-0.002745,...,-0.000264,-0.000426,-7.667659e-07,-0.001598,-0.000148,-0.000099,-0.00066,-0.000055,-0.000055,-0.000055
1,-0.025643,-0.00584,-0.000228,-0.000053,-0.000156,-0.000943,-0.000463,-0.006695,-0.004035,-0.002745,...,-0.000264,-0.000426,-7.667659e-07,-0.001598,-0.000148,-0.000099,-0.00066,-0.000055,-0.000055,-0.000055


In [10]:
tfidf_docs.shape

(4837, 9232)

In [11]:
sms.spam.sum()

638

In [12]:
from sklearn.decomposition import PCA
pca=PCA(n_components=16)
pca=pca.fit(tfidf_docs)
pca_topic_vectors=pca.transform(tfidf_docs)
colums=['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors=pd.DataFrame(pca_topic_vectors,columns=colums,index=index)


In [13]:
pca_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.201,0.003,0.037,0.011,-0.019,-0.053,0.040,-0.065,0.011,-0.080,0.009,0.002,-0.006,-0.028,-0.006,0.039
sms1,0.404,-0.094,-0.078,0.051,0.100,0.047,0.023,0.065,0.023,-0.023,-0.005,0.038,0.036,-0.041,0.053,-0.034
sms2!,-0.030,-0.048,0.090,-0.067,0.091,-0.043,-0.000,-0.001,-0.057,0.051,0.128,0.024,0.019,-0.023,-0.039,0.054
sms3,0.329,-0.033,-0.035,-0.016,0.052,0.056,-0.165,-0.074,0.063,-0.107,0.020,0.029,0.050,-0.071,0.019,-0.074
sms4,0.002,0.031,0.038,0.034,-0.075,-0.093,-0.044,0.061,-0.045,0.029,0.030,-0.012,0.042,0.021,-0.082,-0.024
sms5!,-0.016,0.059,0.014,-0.006,0.122,-0.040,0.005,0.167,-0.023,0.065,0.040,0.057,-0.010,0.090,0.013,0.027


In [14]:
tfidf.vocabulary_
colums_nums,terms=zip(*sorted(zip(tfidf.vocabulary_.values(),tfidf.vocabulary_.keys())))

In [15]:
weight=pd.DataFrame(pca.components_,columns=terms,index=['topic{}'.format(i) for i in range(16)])
pd.options.display.max_columns=8
weight.head(4).round(3)

,!,"""",#,#150,...,…,┾,〨ud,鈥
topic0,-0.071,0.008,-0.001,-0.000,...,-0.002,0.001,0.001,0.001
topic1,0.064,0.008,0.000,-0.000,...,0.003,0.001,0.001,0.001
topic2,0.071,0.027,0.000,0.001,...,0.002,-0.001,-0.001,-0.001
topic3,-0.059,-0.032,-0.001,-0.000,...,0.001,0.000,0.000,0.000


In [16]:
pd.options.display.max_columns=12
deals=weight['! ;) :) half off free crazy deal only $ 80 %'.split()].round(3)*100
deals

,!,;),:),half,off,free,crazy,deal,only,$,80,%
topic0,-7.1,0.1,-0.5,-0.0,-0.4,-2.0,-0.0,-0.1,-2.2,0.3,-0.0,-0.0
topic1,6.4,0.0,7.4,0.1,0.4,-2.3,-0.2,-0.1,-3.8,-0.1,-0.0,-0.2
topic2,7.1,0.2,-0.1,0.0,0.3,4.4,0.1,-0.1,0.7,0.0,0.0,0.1
topic3,-5.9,-0.3,-7.0,0.2,0.3,-0.2,0.0,0.1,-2.3,0.1,-0.1,-0.3
topic4,38.1,-0.1,-12.4,-0.1,-0.2,9.9,0.1,-0.2,3.0,0.3,0.1,-0.1
topic5,-26.5,0.1,-1.5,-0.3,-0.7,-1.3,-0.6,-0.2,-1.8,-0.9,0.0,0.0
topic6,-10.9,-0.5,19.9,-0.4,-0.9,-0.6,-0.2,-0.1,-1.4,-0.0,-0.0,-0.1
topic7,16.1,0.1,-17.9,0.8,0.8,-2.9,0.0,0.1,-1.9,-0.3,0.0,-0.1
topic8,34.3,0.1,5.3,-0.5,-0.5,0.1,-0.4,-0.4,3.1,-0.6,-0.0,-0.2
topic9,7.5,-0.3,16.4,1.4,-0.9,6.4,-0.5,-0.4,3.2,-0.4,-0.0,0.0


In [17]:
deals.T.sum()

topic0    -11.9
topic1      7.6
topic2     12.7
topic3    -15.4
topic4     38.4
topic5    -33.7
topic6      4.8
topic7     -5.2
topic8     40.3
topic9     32.4
topic10   -29.7
topic11    49.9
topic12    25.8
topic13    47.0
topic14    25.5
topic15    -3.1
dtype: float64

In [18]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=16,n_iter=100)
svd_topic_vectors=svd.fit_transform(tfidf_docs.values)

In [19]:
svd_topic_vectors.shape

(4837, 16)

In [20]:
svd_topic_vectors=pd.DataFrame(svd_topic_vectors,columns=colums,index=index)

In [21]:
svd_topic_vectors.round(3).head(5)

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.201,0.003,0.037,0.011,-0.019,-0.053,...,0.007,-0.007,0.002,-0.036,-0.014,0.037
sms1,0.404,-0.094,-0.078,0.051,0.100,0.047,...,-0.004,0.036,0.043,-0.021,0.051,-0.042
sms2!,-0.030,-0.048,0.090,-0.067,0.091,-0.043,...,0.125,0.023,0.026,-0.020,-0.042,0.052
sms3,0.329,-0.033,-0.035,-0.016,0.052,0.056,...,0.022,0.023,0.073,-0.046,0.022,-0.070
sms4,0.002,0.031,0.038,0.034,-0.075,-0.093,...,0.028,-0.009,0.027,0.034,-0.083,-0.021


In [22]:
svd_topic_vectors.T.sum(axis=1)

topic0    -9.592327e-14
topic1    -1.780798e-13
topic2    -1.030287e-13
topic3    -7.238654e-14
topic4     1.679767e-13
topic5     2.393641e-13
topic6     5.861978e-14
topic7    -1.099121e-13
topic8     1.176836e-14
topic9    -3.086420e-13
topic10    2.220446e-15
topic11    3.952394e-14
topic12    7.627232e-14
topic13    7.527312e-14
topic14    3.241851e-14
topic15   -7.549517e-14
dtype: float64

In [23]:
import numpy as np
np.linalg.norm(svd_topic_vectors,axis=1)

array([0.24673077, 0.45489254, 0.23184245, ..., 0.19987923, 0.19586677,
       0.15022858])

In [24]:
import numpy as np
svd_topic_vectors=(svd_topic_vectors.T/np.linalg.norm(svd_topic_vectors,axis=1)).T

In [25]:
svd_topic_vectors.head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.815339,0.011229,0.150911,0.044478,-0.077920,-0.214706,...,0.030397,-0.029076,0.010069,-0.144022,-0.055770,0.149626
sms1,0.888956,-0.206368,-0.170397,0.111870,0.219957,0.103543,...,-0.009661,0.078080,0.095108,-0.045777,0.111280,-0.092359
sms2!,-0.131376,-0.207488,0.389017,-0.289237,0.391411,-0.186100,...,0.541174,0.100243,0.110209,-0.087526,-0.183288,0.225754
sms3,0.778228,-0.077499,-0.081756,-0.037400,0.123524,0.131796,...,0.051789,0.054281,0.171660,-0.108844,0.052625,-0.164697
sms4,0.011327,0.161933,0.201076,0.177766,-0.391843,-0.485564,...,0.147053,-0.045737,0.140988,0.177142,-0.438170,-0.107920
sms5!,-0.062710,0.229681,0.055002,-0.023813,0.479835,-0.157510,...,0.160236,0.214765,-0.143838,0.295326,-0.002431,0.078353


In [26]:
svd_topic_vectors.iloc[:10].dot(svd_topic_vectors.iloc[:10].T).round(1)

,sms0,sms1,sms2!,sms3,sms4,sms5!,sms6,sms7,sms8!,sms9!
sms0,1.0,0.6,-0.1,0.6,-0.0,-0.3,-0.3,-0.1,-0.3,-0.3
sms1,0.6,1.0,-0.2,0.8,-0.2,0.0,-0.2,-0.2,-0.1,-0.1
sms2!,-0.1,-0.2,1.0,-0.2,0.1,0.4,0.0,0.3,0.5,0.4
sms3,0.6,0.8,-0.2,1.0,-0.2,-0.3,-0.1,-0.3,-0.2,-0.1
sms4,-0.0,-0.2,0.1,-0.2,1.0,0.2,0.0,0.1,-0.4,-0.2
sms5!,-0.3,0.0,0.4,-0.3,0.2,1.0,-0.1,0.1,0.3,0.4
sms6,-0.3,-0.2,0.0,-0.1,0.0,-0.1,1.0,0.1,-0.2,-0.2
sms7,-0.1,-0.2,0.3,-0.3,0.1,0.1,0.1,1.0,0.1,0.4
sms8!,-0.3,-0.1,0.5,-0.2,-0.4,0.3,-0.2,0.1,1.0,0.3
sms9!,-0.3,-0.1,0.4,-0.1,-0.2,0.4,-0.2,0.4,0.3,1.0


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)
counter=CountVectorizer(tokenizer=casual_tokenize)
bow_docs=pd.DataFrame(counter.fit_transform(raw_documents=sms.text).toarray(),index=index)

In [28]:
colums_nums,terms=zip(*sorted(zip(counter.vocabulary_.values(),counter.vocabulary_.keys())))
bow_docs.columns=terms

In [29]:
len(terms)

9232

In [30]:
for item in sms.loc['sms0'].text.split():
    print(item)

Go
until
jurong
point,
crazy..
Available
only
in
bugis
n
great
world
la
e
buffet...
Cine
there
got
amore
wat...


In [31]:
bow_docs.head(5)

,!,"""",#,#150,#5000,$,...,’,“,…,┾,〨ud,鈥
sms0,0,0,0,0,0,0,...,0,0,0,0,0,0
sms1,0,0,0,0,0,0,...,0,0,0,0,0,0
sms2!,0,0,0,0,0,0,...,0,0,0,0,0,0
sms3,0,0,0,0,0,0,...,0,0,0,0,0,0
sms4,0,0,0,0,0,0,...,0,0,0,0,0,0


In [32]:
bow_docs.loc['sms0'][bow_docs.loc['sms0']>0].head()

,            1
..           1
...          2
amore        1
available    1
Name: sms0, dtype: int64

In [33]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia=LDiA(n_components=16,learning_method='batch')
ldia=ldia.fit(bow_docs)
ldia.components_.shape

(16, 9232)

In [34]:
pd.set_option('display.width',75)
components=pd.DataFrame(ldia.components_.T,index=terms,columns=colums)

In [35]:
components.round(3).head(3)

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
!,184.034,15.001,72.224,394.952,45.481,36.135,...,37.424,44.181,64.397,297.295,41.161,11.703
"""",0.681,4.222,2.413,0.063,152.351,0.063,...,8.421,11.419,0.065,62.717,12.269,0.063
#,0.063,0.063,0.063,0.063,0.063,2.065,...,0.063,0.063,1.068,4.055,0.063,0.063


In [36]:
components.topic3.sort_values(ascending=False)[:10]

!       394.952246
.       218.049724
to      119.533134
u       118.857546
call    111.948541
£       107.358914
,        96.954384
*        90.314783
your     90.215961
is       75.750037
Name: topic3, dtype: float64

In [37]:
ldia16_topic_vector=ldia.transform(bow_docs)
ldia16_topic_vector=pd.DataFrame(ldia16_topic_vector,index=index,columns=colums)

In [38]:
ldia16_topic_vector.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.00,0.62,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00
sms1,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.12,0.01,0.01,0.01,0.01
sms2!,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00
sms3,0.00,0.00,0.00,0.00,0.09,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00
sms4,0.39,0.00,0.33,0.00,0.00,0.00,...,0.00,0.00,0.09,0.00,0.00,0.00


In [39]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(ldia16_topic_vector,sms.spam,test_size=0.5,random_state=271828)

In [40]:
lda=LDA(n_components=1)
ida=lda.fit(x_train,y_train)
sms['ldia16_spam']=lda.predict(ldia16_topic_vector)

In [41]:
round(float(lda.score(x_test,y_test)),2)

0.94

In [42]:
x_test

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
sms2221,0.006250,0.006250,0.006250,0.006250,0.006250,0.392123,...,0.006250,0.006250,0.520377,0.006250,0.006250,0.006250
sms2934,0.002976,0.002976,0.002976,0.002976,0.002976,0.002976,...,0.002976,0.002976,0.152411,0.805922,0.002976,0.002976
sms2794,0.004808,0.178790,0.297406,0.004808,0.004808,0.156295,...,0.004808,0.004808,0.004808,0.309817,0.004808,0.004808
sms412,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944,...,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944
sms1187!,0.001894,0.001894,0.001894,0.186246,0.001894,0.001894,...,0.001894,0.001894,0.001894,0.177872,0.001894,0.001894
...,...,...,...,...,...,...,...,...,...,...,...,...,...
sms3946,0.004167,0.004167,0.004167,0.004167,0.004167,0.937500,...,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167
sms3872,0.006944,0.006944,0.565757,0.006944,0.006944,0.006944,...,0.006944,0.006944,0.188976,0.006944,0.006944,0.006944
sms4332,0.002404,0.002404,0.245351,0.002404,0.002404,0.002404,...,0.002404,0.002404,0.162163,0.002404,0.002404,0.002404
sms3105,0.001330,0.001330,0.001330,0.001330,0.001330,0.319720,...,0.001330,0.001330,0.001330,0.314669,0.001330,0.001330


In [43]:
y_test

sms2221     0
sms2934     0
sms2794     0
sms412      0
sms1187!    1
           ..
sms3946     0
sms3872     0
sms4332     0
sms3105     0
sms4671     0
Name: spam, Length: 2419, dtype: int64

In [44]:
len(y_test)

2419

In [45]:
len(x_train)

2418